In [ ]:
%pip install pandas
%pip install scikit-learn
%pip install plotly
%pip install numpy
%pip install matplotlib
%pip install nbformat
%pip install opencv-python
%pip install tensorflow
%pip install wandb

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import os
import plotly.express as px

from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.metrics import F1Score, CategoricalAccuracy
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import wandb
%env "WANDB_NOTEBOOK_NAME" "model_balanced.ipynb"

wandb.login()

from wandb.integration.keras import WandbMetricsLogger, WandbModelCheckpoint

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


env: "WANDB_NOTEBOOK_NAME"="model_balanced.ipynb"


wandb: Currently logged in as: pedro-mariani (dspro2-group9). Use `wandb login --relogin` to force relogin


In [2]:
label_df = pd.read_csv(
    "../../../data/processed/combined/combined_label.csv", index_col=False
)
data_dir = "../../../data/processed/combined/img/"

label_df.head()

,image_name,expression_label
0,angry_0.jpg,angry
1,angry_1.jpg,angry
2,fear_2.jpg,fear
3,angry_4.jpg,angry
4,angry_5.jpg,angry


In [3]:
def load_data(dir_path, img_size):
    X = []
    y = []

    for index, row in label_df.iterrows():
        image_path = os.path.join(dir_path, row["image_name"])
        img = cv2.imread(image_path)
        img = (
            cv2.resize(img, img_size, interpolation=cv2.INTER_AREA).astype("float32")
            / 255
        )
        X.append(img)
        y.append(row["expression_label"])

    X = np.array(X)
    y = np.array(y)
    return X, y

In [4]:
IMG_SIZE = (48, 48)

X, y = load_data(data_dir, IMG_SIZE)

In [5]:
dummies = pd.get_dummies(label_df["expression_label"])
dummies.head()

,angry,disgust,fear,happy,neutral,sad,surprise
0,True,False,False,False,False,False,False
1,True,False,False,False,False,False,False
2,False,False,True,False,False,False,False
3,True,False,False,False,False,False,False
4,True,False,False,False,False,False,False


In [6]:
y = dummies.values
y

array([[ True, False, False, ..., False, False, False],
       [ True, False, False, ..., False, False, False],
       [False, False,  True, ..., False, False, False],
       ...,
       [ True, False, False, ..., False, False, False],
       [ True, False, False, ..., False, False, False],
       [False, False,  True, ..., False, False, False]])

In [7]:
train_ratio = 0.80
test_ratio = 0.10
validation_ratio = 0.10


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_ratio)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train, y_train, test_size=validation_ratio / (train_ratio + test_ratio)
)

print(X_train.shape)
print(X_valid.shape)
print(X_test.shape)

print(y_train.shape)
print(y_valid.shape)
print(y_test.shape)

(63451, 48, 48, 3)
(7932, 48, 48, 3)
(7932, 48, 48, 3)
(63451, 7)
(7932, 7)
(7932, 7)


# Distribution of classes in each set


In [8]:
class_count_in_train_set = np.sum(y_train, axis=0)
# dropped_class_count_in_train_set = sum(
#     1 for sublist in y_train if all(element == 0 for element in sublist)
# )
# class_count_in_train_set = np.append(
#     class_count_in_train_set, dropped_class_count_in_train_set
# )

print("Class distribution:")
print(class_count_in_train_set)

Class distribution:
[ 6931  3639  4959 12650 13099 13313  8860]


In [9]:
labels = ["angry", "disgust", "fear", "happy", "neutral", "sad", "surprise"]
fig = px.bar(x=labels, y=class_count_in_train_set, color=labels)

fig.update_layout(
    title="Distribution of each label in train set",
    xaxis_title="Expressions",
    yaxis_title="Count",
    height=600,
    width=800,
)

fig.update_traces(texttemplate="%{y}", textposition="inside")

fig.show()

In [10]:
class_count_in_validation_set = np.sum(y_valid, axis=0)
# dropped_class_count_in_validation_set = sum(
#     1 for sublist in y_valid if all(element == 0 for element in sublist)
# )
# class_count_in_validation_set = np.append(
#     class_count_in_validation_set, dropped_class_count_in_validation_set
# )

print("Class distribution:")
print(class_count_in_validation_set)

Class distribution:
[ 890  412  621 1558 1696 1667 1088]


In [11]:
labels = ["angry", "disgust", "fear", "happy", "neutral", "sad", "surprise"]
fig = px.bar(x=labels, y=class_count_in_validation_set, color=labels)

fig.update_layout(
    title="Distribution of each label in validation set",
    xaxis_title="Expressions",
    yaxis_title="Count",
    height=600,
    width=800,
)

fig.update_traces(texttemplate="%{y}", textposition="inside")

fig.show()

In [12]:
# Assuming your 2D array is named 'encoded_data'
class_count_in_test_set = np.sum(y_test, axis=0)
# dropped_class_count_in_test_set = sum(
#     1 for sublist in y_test if all(element == 0 for element in sublist)
# )
# class_count_in_test_set = np.append(
#     class_count_in_test_set, dropped_class_count_in_test_set
# )

print("Class distribution:")
print(class_count_in_test_set)

Class distribution:
[ 803  491  629 1602 1637 1656 1114]


In [13]:
labels = ["angry", "disgust", "fear", "happy", "neutral", "sad", "surprise"]
fig = px.bar(x=labels, y=class_count_in_test_set, color=labels)

fig.update_layout(
    title="Distribution of each label in test set",
    xaxis_title="Expressions",
    yaxis_title="Count",
    height=600,
    width=800,
)

fig.update_traces(texttemplate="%{y}", textposition="inside")

fig.show()

# Data augmentation to balance the dataset


In [14]:
# Assume x_train and y_train are your original data
# Create an ImageDataGenerator with desired augmentations
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    brightness_range=(0.8, 1.2),
    zoom_range=0.1,
)

# Generate augmented images for the minority class
angry_indices = np.where(y_train[:, 0] == 1)[0]
disgust_indices = np.where(y_train[:, 1] == 1)[0]
fear_indices = np.where(y_train[:, 2] == 1)[0]
augmented_angry_images = datagen.flow(
    X_train[angry_indices], y_train[angry_indices], batch_size=len(angry_indices)
)
augmented_disgust_images = datagen.flow(
    X_train[disgust_indices], y_train[disgust_indices], batch_size=len(disgust_indices)
)
augmented_fear_images = datagen.flow(
    X_train[fear_indices], y_train[fear_indices], batch_size=len(fear_indices)
)

# Combine augmented minority class images with original data
x_train_balanced = np.concatenate(
    [
        X_train,
        augmented_angry_images[0][0],
        augmented_disgust_images[0][0],
        augmented_fear_images[0][0],
    ]
)
y_train_balanced = np.concatenate(
    [
        y_train,
        augmented_angry_images[0][1],
        augmented_disgust_images[0][1],
        augmented_fear_images[0][1],
    ]
)

In [18]:
class_count_in_balanced_train_set = np.sum(y_train_balanced, axis=0)

print("Class distribution:")
print(class_count_in_balanced_train_set)

Class distribution:
[13862  7278  9918 12650 13099 13313  8860]


In [19]:
labels = ["angry", "disgust", "fear", "happy", "neutral", "sad", "surprise"]
fig = px.bar(x=labels, y=class_count_in_balanced_train_set, color=labels)

fig.update_layout(
    title="Distribution of each label in balanced train set",
    xaxis_title="Expressions",
    yaxis_title="Count",
    height=600,
    width=800,
)

fig.update_traces(texttemplate="%{y}", textposition="inside")

fig.show()

# Model training


In [20]:
IMG_SIZE = (48, 48)

base_model = VGG16(weights="imagenet", include_top=False, input_shape=IMG_SIZE + (3,))
for layer in base_model.layers[:-1]:
    layer.trainable = False

In [24]:
NUM_CLASSES = 7

model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(1000, activation="relu"))
model.add(Dropout(0.4))
model.add(Dense(NUM_CLASSES, activation="softmax"))

In [25]:
wandb.init(
    # set the wandb project where this run will be logged
    project="Emotion Recognition",
    # track hyperparameters and run metadata with wandb.config
    config={
        "optimizer": "adam",
        "loss": "categorical_crossentropy",
        "metric": ["categorical_accuracy"],
        "epoch": 4,
        "batch_size": 32,
    },
)

In [26]:
config = wandb.config


model.compile(
    loss=config["loss"],
    optimizer=config["optimizer"],
    metrics=config["metric"],
)

history = model.fit(
    x_train_balanced,
    y_train_balanced,
    validation_data=(X_valid, y_valid),
    epochs=config["epoch"],
    batch_size=config["batch_size"],
    callbacks=[WandbMetricsLogger(log_freq=5)],
)


wandb.finish()

Epoch 1/4


wandb: ERROR Unable to log learning rate.


2469/2469 ━━━━━━━━━━━━━━━━━━━━ 185s 75ms/step - categorical_accuracy: 0.3225 - loss: 1.6444 - val_categorical_accuracy: 0.3174 - val_loss: 1.6846
Epoch 2/4
2469/2469 ━━━━━━━━━━━━━━━━━━━━ 185s 75ms/step - categorical_accuracy: 0.3502 - loss: 1.5715 - val_categorical_accuracy: 0.3433 - val_loss: 1.6576
Epoch 3/4
2469/2469 ━━━━━━━━━━━━━━━━━━━━ 183s 74ms/step - categorical_accuracy: 0.3627 - loss: 1.5526 - val_categorical_accuracy: 0.3456 - val_loss: 1.6512
Epoch 4/4
2469/2469 ━━━━━━━━━━━━━━━━━━━━ 185s 75ms/step - categorical_accuracy: 0.3688 - loss: 1.5395 - val_categorical_accuracy: 0.3506 - val_loss: 1.6437


batch/batch_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
batch/categorical_accuracy,▁▃▄▅▅▅▅▅▅▆▅▆▇▇▇▇▇▇▇▇▇▇█▇█▇▇▇▇▇██████████
batch/loss,█▅▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/categorical_accuracy,▁▅▇█
epoch/epoch,▁▃▆█
epoch/loss,█▄▂▁
epoch/val_categorical_accuracy,▁▆▇█
epoch/val_loss,█▃▂▁
batch/batch_step,9875
batch/categorical_accuracy,0.36883
batch/loss,1.5393
